# Intento 1 (_holdout_)

| Score | Estimación _holdout_ | Timestamp |
| --- | --- | --- |
| **0.8589** | 0.8622 | 2021-12-31 02:08:12 UTC |

- Uso de LightGBM.
- Estrategia de preprocesado básica.

In [87]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.multioutput import MultiOutputClassifier
import lightgbm as lgb

# código común
import common.preprocessing as pre
from common.vars import SEED

# display all columns
pd.options.display.max_columns = None

In [88]:
train_features = pd.read_csv('../data/flu_training_set_features.csv', index_col='respondent_id')
train_labels = pd.read_csv('../data/flu_training_set_labels.csv', index_col='respondent_id')
test_features = pd.read_csv('../data/flu_test_set_features.csv', index_col='respondent_id')

In [89]:
def preprocess(df_train, df_test, pre_onehot=True):
  """PREPROCESAMIENTO
  """
  _df_train = df_train.copy()
  _df_test = df_test.copy()
  # antes de realizar one-hot encoding, eliminamos los valores perdidos de atributos categóricos
  # lo haremos imputando a la clase mayoritaria (mode)
  if pre_onehot:
    attrs_imp = ['race', 'employment_status', 'census_msa', 'hhs_geo_region', 'employment_industry', 'employment_occupation']
    _df_train = pre.impute(_df_train, strategy='mode', attrs=attrs_imp)
    _df_test = pre.impute(_df_test, strategy='mode', attrs=attrs_imp)
  
  # discretizamos estas columnas asignando un valor discreto cualquiera, secuencialmente
  # (hacemos la labor de LabelEncoder para los atributos)
  for attr in df_train.columns:
    _df_train, discrete_dict = pre.trans_discretize(_df_train, attr)
    # añadimos las mismas claves que df_train
    _df_test = pre.trans_discretize(_df_test, attr, discrete_dict)
  
  # VALORES PERDIDOS
  # del resto de atributos, imputamos la mediana (median)
  _df_train = pre.impute(_df_train, strategy='median')
  _df_test = pre.impute(_df_test, strategy='median')
  
  return _df_train, _df_test

In [90]:
train_features, test_features = preprocess(train_features, test_features)

In [98]:
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)

def cross_validation(model, X, y, cv):
  measures = []
  time_total_0 = time.time()

  for train, test in cv.split(X, y):
    X_train = X.iloc[train, :].values
    X_test = X.iloc[test, :].values
    y_train = y.iloc[train, :].values
    y_test = y.iloc[test, :].values
    time_iter_0 = time.time()
    _model = model.fit(X_train, y_train)
    time_iter = time.time() - time_iter_0
    preds = _model.predict_proba(X_test)
    y_preds = pd.DataFrame(
      {
        'h1n1_vaccine':     preds[0][:,1],
        'seasonal_vaccine': preds[1][:,1],
      },
      index = y.iloc[test,:].index
    )
    measure = roc_auc_score(y_test, y_preds)
    print(f"[CV] AUC score: {measure:.4f}. Time: {time_iter:6.2f}")
    measures.append(measure)
  
  time_total = time.time() - time_total_0
  print(f"[CV] Average AUC: {np.mean(measures):.4f}. Total time: {time_total:6.2f}")
  return model, y_preds

def preds_to_df(preds, features):
  return pd.DataFrame(
    {
      'h1n1_vaccine': preds[0][:,1],
      'seasonal_vaccine': preds[1][:,1]
    },
    index = features.index
  )

### _Holdout_ como proxy de ROC en test

In [92]:
# split train features in two
X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.2, random_state=SEED)

In [93]:
# LightGBM
lgbm = lgb.LGBMClassifier(objective='binary', n_estimators=200)
multi_lgbm = MultiOutputClassifier(lgbm)
lgbm_cvd, y_preds_lgbm = cross_validation(multi_lgbm, X_train, y_train, kf)

[CV] AUC score: 0.8631. Time:   0.83
[CV] AUC score: 0.8652. Time:   1.07
[CV] AUC score: 0.8544. Time:   1.05
[CV] AUC score: 0.8494. Time:   1.11
[CV] AUC score: 0.8679. Time:   1.17
[CV] Average AUC: 0.8600. Total time:   5.72


In [94]:
# aprendemos con todos los ejemplos
lgbm_cvd = lgbm_cvd.fit(X_train, y_train)
# aplicamos probabilidad
lgbm_preds = lgbm_cvd.predict_proba(X_test)

In [97]:
roc_auc_score(y_test, preds_to_df(lgbm_preds, X_test))

0.8621941060149647

### Submission

In [99]:
# LightGBM
lgbm = lgb.LGBMClassifier(objective='binary', n_estimators=200)
multi_lgbm = MultiOutputClassifier(lgbm)
lgbm_cvd, y_preds_lgbm = cross_validation(multi_lgbm, train_features, train_labels, kf)

[CV] AUC score: 0.8622. Time:   1.82
[CV] AUC score: 0.8638. Time:   2.00
[CV] AUC score: 0.8614. Time:   1.71
[CV] AUC score: 0.8648. Time:   1.99
[CV] AUC score: 0.8610. Time:   1.68
[CV] Average AUC: 0.8626. Total time:   9.96


In [100]:
# aprendemos con todos los ejemplos
lgbm_cvd = lgbm_cvd.fit(train_features, train_labels)
# aplicamos probabilidad
lgbm_preds = lgbm_cvd.predict_proba(test_features)

In [101]:
#preds_to_df(lgbm_preds, test_features).to_csv('../submissions/01.csv', index=False)